## Behind the pipeline

In [3]:
# pipeline code example
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

classifier([
    'I have been waiting for hugging face course my whole life',
    'I hate this so much'
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\arman\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\arman\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[{'label': 'POSITIVE', 'score': 0.9987958669662476},
 {'label': 'NEGATIVE', 'score': 0.9995144605636597}]

# Tokenizer example

In [3]:
from transformers import AutoTokenizer

checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
raw_inputs = [
    'I have been waiting for hugging face course my whole life',
    'I hate this so much'
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[  101,  1045,  2031,  2042,  3403,  2005, 17662,  2227,  2607,
         2026,  2878,  2166,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   102,     0,     0,
            0,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(2, 13), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])>}


# Model Example

In [1]:
from transformers import TFAutoModel

checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [5]:
outputs = model(inputs)
print(outputs)
print(outputs.last_hidden_state.shape)

TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(2, 13, 768), dtype=float32, numpy=
array([[[ 0.10004615,  0.08926946,  0.6018838 , ..., -0.04704958,
          0.5371728 , -0.39642936],
        [ 0.49409273,  0.4537986 ,  0.3672404 , ...,  0.12324937,
          0.6759264 , -0.24971434],
        [ 0.564472  ,  0.51474637,  0.17395115, ...,  0.02975993,
          0.5080044 , -0.21472351],
        ...,
        [ 0.5826182 ,  0.23295236,  0.35156295, ..., -0.22303683,
          0.70681447, -0.45775637],
        [ 0.561423  ,  0.38966078,  0.37009525, ..., -0.11075118,
          0.5787301 , -0.3207711 ],
        [ 0.6985472 ,  0.21465364,  0.3985237 , ...,  0.42147446,
          0.34449214, -0.9425872 ]],

       [[-0.30882236,  0.7331757 , -0.18604988, ..., -0.13053825,
         -0.93600386, -0.04329733],
        [-0.33396998,  0.9830432 , -0.09462059, ..., -0.3824745 ,
         -0.61756295,  0.20076862],
        [-0.1687141 ,  0.8781159 , -0.11165855, ..., -0.23800479,
         -0.7

In [6]:
## example huggingface architecture
from transformers import TFAutoModelForSequenceClassification
checkpoint="distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)
print(outputs.logits.shape)


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


(2, 2)


In [7]:
print(outputs.logits)

tf.Tensor(
[[-3.2831624  3.4376247]
 [ 4.2140603 -3.4158013]], shape=(2, 2), dtype=float32)


In [8]:
## Those are not probabilities but logits, the raw, unnormalized scores outputted by the last layer of the model. To be converted to probabilities, they need to go through a SoftMax layer

import tensorflow as tf
predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[1.2041373e-03 9.9879587e-01]
 [9.9951446e-01 4.8549214e-04]], shape=(2, 2), dtype=float32)


In [9]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

## 3.Models

In [10]:
# Creating a transformer
from transformers import BertConfig, TFBertModel

# Building the config
config = BertConfig()

# Building the model from the config
# In this way of Creating a model from the default configuration initializes it with random values
# The model can be used in this state, but it will output gibberish; it needs to be trained first.
model = TFBertModel(config)

# The configuration contains many attributes that are used to build the model
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.32.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
# Loading a Transformer model that is already trained
from transformers import TFBertModel
model = TFBertModel.from_pretrained("bert-base-cased")
